<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.01/colab/02_IR_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR - Assignment 1

## Part 2 - DF IDF

In this part we will use the TF-IDF with BM25 and Okapi Algorithm

In [1]:
# get the cleaned data from the source

aj_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/A_J_word.csv"
bbc_wrod_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/BBC_word.csv"
jp_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/J_P_word.csv"
nyt_word_link = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/NYT_word.csv"

In [ ]:
# download the data using http
